# [verbose](https://buillee.tistory.com/143)

# 정확히 이해되지 않은 개념

+ 사용자 정의 객체

# 모델의 저장과 복원

훈련하는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있습니다. 모델이 중지된 지점부터 다시 훈련할 수 있어 한 번에 오랫동안 훈련하지 않아도 됩니다. 모델을 저장하면 다른 사람에게 공유할 수 있고 작업을 재현할 수 있습니다. 연구한 모델과 기법을 공개할 때 많은 머신 러닝 기술자들이 다음과 같은 것들을 제공합니다.

+ 모델을 만드는 코드
+ 모델의 훈련된 가중치 또는 파라미터

이런 데이터를 공유하면 다른 사람들이 모델의 작동 방식을 이해하고 새로운 데이터로 모델을 실험하는데 도움이 됩니다.

주의 : 신뢰할 수 없는 코드는 조심하세요. 자세한 내용은 [텐서플로를 안전하게 사용하기](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) 문서를 참고하세요.

### 저장 방식

사용하는 API에 따라서 여러가지 방법으로 텐서플로우 모델을 저장할 수 있습니다. 이번 튜토리얼에서는 고수준 API인 tf.keras를 사용해서 저장하고 복원하는 방식에 대해 알아보겠습니다.

## 1. 설정

### 설치와 임포트

필요한 라이브러리를 설치하고 텐서플로우를 임포트합니다.

In [1]:
!pip install pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다.

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

### 예제 데이터셋 받기

MNIST 데이터셋으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠습니다. 모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 1,000개만 사용하겠습니다.

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


### 모델 정의

먼저 간단한 모델을 하나 만들어 보겠습니다.

In [4]:
def create_model():
    model = keras.Sequential([
                              keras.layers.Dense(512, activation='relu', input_shape=(784,)),
                              keras.layers.Dropout(0.2),
                              keras.layers.Dense(10)
    ])

    model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 2. 훈련하는 동안 체크포인트 저장하기

훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장하도록 하는 것이 많이 사용하는 방법입니다. 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있습니다.

`keras.callbacks.ModelCheckpoint`은 이런 작업을 수행하는 콜백(callback)입니다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공합니다.

### 체크포인트 콜백 사용하기

훈련하는 동안 가중치를 저장하기 위해 `ModelCheckpoint` 콜백을 만들어 보겠습니다.

In [5]:
checkpoint_path = "training_1/cp.ckpt"  # 체크 포인트 파일 위치(training_1 폴더 안에 파일 이름 cp.ckpt)
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)
# verbose = 0 : 아무것도 띄우지 않는다. verbose = 1 : [========] 바 생성. verbose = 2 : 에포크마다 한 줄에 하나씩 값 표시

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, train_labels, epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다.

Epoch 1/10
32/32 [==============================] - 1s 12ms/step - loss: 1.1657 - accuracy: 0.6500 - val_loss: 0.7394 - val_accuracy: 0.7890

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.4269 - accuracy: 0.8720 - val_loss: 0.5539 - val_accuracy: 0.8240

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2895 - accuracy: 0.9340 - val_loss: 0.4669 - val_accuracy: 0.8540

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2159 - accuracy: 0.9450 - val_loss: 0.4395 - val_accuracy: 0.8660

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 0.1501 - accuracy: 0.9640 - val_loss: 0.4268 - val_accuracy: 0.8650

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s

이 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다.

만들어진 체크포인트를 확인해 봅시다.

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


훈련하지 않은 새로운 모델을 만들고 이전에 저장했던 가중치를 로드해서 성능을 평가해 봅시다.

가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 합니다. 여기서는 동일한 구조로 모델을 만들었으므로 다른 *객체*이지만 가중치를 공유할 수 있습니다.

훈련하지 않은 새 모델을 만들고 이전에 저장했던 가중치를 로드하기 전 테스트 세트만 가지고 평가해 봅시다.

In [7]:
# 기본 모델 객체를 만듭니다.
model = create_model()
# 모델을 평가합니다.
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3798 - accuracy: 0.0600
훈련되지 않은 모델의 정확도:  6.00%


새로 만든 모델은 훈련이 된 데이터가 없으므로 테스트 세트로 평가하면 정확도가 상당히 낮은 것을 볼 수 있습니다.

체크포인트에서 가중치를 로드하고 다시 평가해 봅시다.

In [8]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4166 - accuracy: 0.8630
복원된 모델의 정확도: 86.30%


### 체크포인트 콜백 매개변수

이 콜백 함수는 몇 가지 매개변수를 제공합니다. 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있습니다.

In [9]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
# epoch:04d 는 4자리의 정수형태로 에포크를 출력하겠다는 의미(빈자리는 0으로 채운다)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다.
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

model = create_model()
# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다.
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images, train_labels, epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


만들어진 체크포인트를 확인해 봅시다.

In [10]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


마지막 체크포인트를 선택해보겠습니다.

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

텐서플로우는 기본적으로 최근 5개의 체크포인트만 저장합니다.

모델을 초기화하고 최근 체크포인트를 로드하여 테스트해 보겠습니다.

In [12]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4978 - accuracy: 0.8730
복원된 모델의 정확도: 87.30%


## 3. 수동으로 가중치 저장하기

수동으로 가중치를 저장하는 것도 쉽습니다. `Model.save_weights` 메서드를 사용합니다. `tf.keras`의 `save_weights`는 .ckpt 확장자를 가진 텐서플로우 체크포인트 포멧을 사용합니다.(.h5 확장자의 HDF5로 저장하는 것은 이후에 다룹니다)

In [13]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4978 - accuracy: 0.8730
복원된 모델의 정확도: 87.30%


## 4. 전체 모델 저장하기

`model.save` 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다. 모델을 저장하기 때문에 원본 파이썬 코드가 없어도 사용할 수 있습니다. 정확히 중지한 시점에서 다시 훈련을 시작할 수 있습니다.

전체 모델을 저장하는 기능은 매우 유용합니다. TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다. 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수도 있습니다.

### SavedModel 포맷

SavedModel 포맷은 모델을 직렬화하는 방법입니다. 이 포맷으로 저장한 모델은 `keras.models.load_model`로 복원할 수 있고 텐서플로 서빙과 호환됩니다.

[SavedModel 가이드](https://www.tensorflow.org/guide/saved_model)에서 SavedModel을 서빙하고 점검하는 자세한 방법을 제공합니다.

In [14]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model  # 'saved_model'이라는 이름의 폴더를 만듭니다.
model.save('saved_model/my_model')  # my_model이라는 이름으로 저장합니다.

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1651 - accuracy: 0.6620
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4206 - accuracy: 0.8920
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2858 - accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2130 - accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1598 - accuracy: 0.9640
INFO:tensorflow:Assets written to: saved_model/my_model/assets


저장된 모델의 디렉토리를 확인해 보겠습니다.

In [15]:
!ls saved_model

!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


저장된 모델로부터 새로운 케라스 모델을 로드합니다.

In [16]:
new_model = keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일 되어 있습니다. 이 모델을 평가하고 예측해 보겠습니다.

In [17]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4534 - accuracy: 0.8540
복원된 모델의 정확도: 85.40%
(1000, 10)


### HDF5 파일로 저장하기

keras는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [18]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

new_model = keras.models.load_model('my_model.h5')

new_model.summary()

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1914 - accuracy: 0.6620
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4389 - accuracy: 0.8660
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2958 - accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2027 - accuracy: 0.9570
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1577 - accuracy: 0.9640
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable

In [19]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4296 - accuracy: 0.8690
복원된 모델의 정확도: 86.90%


`model.save` 메서드는 모델 구조를 파악하여 저장합니다. 아래의 항목들을 저장합니다.

+ 가중치 값
+ 모델 구조
+ 모델의 훈련 설정(컴파일할 때 전달한 값)
+ 옵티마이저와 옵티마이저 상태(필요하다면)(중지된 지점부터 다시 시작할 수 있습니다)

체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(`tf.compat.v1.train`)를 저장할 수 없습니다. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다. 따라서 옵티마이저의 상태를 잃게 됩니다.

### 사용자 정의 객체

SavedModel 포맷을 사용한다면 이 섹션은 건너 뛰어도 됩니다. HDF5와 SavedModel 사이의 주요 차이점은 HDF5가 객체로 모델 구조를 저장하고 SavedModel은 실행 그래프로 저장하는 것입니다. 따라서 SavedModel은 원본 코드 없이 상속 모델과 사용자 정의 층 같은 사용자 정의 객체를 저장할 수 있습니다.

사용자 정의 객체를 HDF5로 저장하려면 다음 과정을 따르시길 바랍니다.

1. 객체에 `get_config` 메서드를 정의하고 선택적으로 `from_config` 클래스 메서드를 정의합니다.
    * `get_config(self)`는 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환합니다.
    * `from_config(cls, config)`는 `get_config`에서 반환된 설정을 사용해 새로운 객체를 만듭니다. 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(`return cls(**config)`).
2. 모델을 로드할 때 이 객체를 `custom_objects` 매개변수로 전달합니다. 문자열 클래스 이름과 파이썬 클래스를 매핑한 딕서너리를 매개변수로 제공해야 합니다. 예를 들면 `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

사용자 정의 객체와 `get_config`에 관한 예제를 보려면 [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) 튜토리얼을 참고하세요. 